In [2]:
import cv2
import numpy as np

In [3]:
def show_image(image_to_show):
    cv2.imshow('{}'.format(image_to_show.shape), image_to_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
class Resize(object):
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, X):
        _X = cv2.resize(X, self.output_size)
        w, h = self.output_size
        return _X
    
    
class Clip(object):
    def __init__(self, mini, maxi=None):
        if maxi is None:
            self.mini, self.maxi = 0, mini
        else:
            self.mini, self.maxi = mini, maxi

    def __call__(self, X):
        mini_mask = np.where(X < self.mini)
        maxi_mask = np.where(X > self.maxi)
        X[mini_mask] = self.mini
        X[maxi_mask] = self.maxi
        return X
    
    
class Normalize(object):
    def __init__(self, axis=None):
        self.axis = axis

    def __call__(self, X):
        mini = np.min(X, self.axis)
        maxi = np.max(X, self.axis)
        X = (X - mini) / (maxi - mini)
        return X

    
class Standardize(object):
    def __init__(self, axis=None):
        self.axis = axis

    def __call__(self, X):
        mean =  np.mean(X, self.axis)
        std = np.std(X, self.axis)
        X = (X - mean) / std
        return X   
    
    
class Flip(object):
    def __call__(self, X):
        for axis in [0, 1]:
            if np.random.rand(1) < 0.5:
                X = np.flip(X, axis)
        return X
    
    
class Crop(object):
    def __init__(self, min_size_ratio, max_size_ratio=(1, 1)):
        self.min_size_ratio = np.array(list(min_size_ratio))
        self.max_size_ratio = np.array(list(max_size_ratio))

    def __call__(self, X):
        size = np.array(X.shape[:2])
        mini = self.min_size_ratio * size
        maxi = self.max_size_ratio * size

        h = np.random.randint(mini[0], maxi[0])
        w = np.random.randint(mini[1], maxi[1])

        shift_h = np.random.randint(0, size[0] - h)
        shift_w = np.random.randint(0, size[1] - w)
        X = X[shift_h:shift_h+h, shift_w:shift_w+w]

        return X
    

class Sharpen(object):
    def __init__(self, max_center=20):
        self.identity = np.array([[0, 0, 0],
                                  [0, 1, 0],
                                  [0, 0, 0]])
        self.sharpen = np.array([[ 0, -1,  0],
                                 [-1,  4, -1],
                                 [ 0, -1,  0]]) / 4
        self.max_center = max_center

    def __call__(self, X):

        sharp = self.sharpen * np.random.random() * self.max_center
        kernel = self.identity + sharp

        X = cv2.filter2D(X, -1, kernel)
        return X
    
    
class GaussianBlur(object):
    def __init__(self, max_kernel=(7, 7)):
        self.max_kernel = max_kernel

    def __call__(self, X):
        kernel_size = (
            np.random.randint(1, self.max_kernel[0]) * 2 + 1,
            np.random.randint(1, self.max_kernel[1]) * 2 + 1,
        )
        X = cv2.GaussianBlur(X, kernel_size, 0)
        return X
    
    
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


class UniformNoise(object):
    def __init__(self, low=-50, high=50):
        self.low = low
        self.high = high

    def __call__(self, X):
        noise = np.random.randint(self.low, self.high, X.shape)
        noise = noise.astype(int)
        X = X + noise
        X[X < 0] = 0
        X[X > 255] = 255
        return X
    
    
class GaussianNoise(object):
    def __init__(self, center=0, std=50):
        self.center = center
        self.std = std

    def __call__(self, X):
        noise = np.random.normal(self.center, self.std, X.shape)
        X = X + noise
        X[X < 0] = 0
        X[X > 255] = 255
        return X
    
    
def rotate_func(im_1, angle):
    h, w = im_1.shape[:2]
    center = (w/2, h/2)
    rotate_matrix = cv2.getRotationMatrix2D(center, angle=angle, scale=1)
    return cv2.warpAffine(im_1, rotate_matrix, dsize=(w, h))



In [5]:
im_1 = cv2.imread('123.jpg')

show_image(im_1)

In [ ]:
outp_size = [200, 200]

resize_ = Resize(outp_size)

im_1_r = resize_.__call__(im_1)

In [ ]:
min_ = 0
max_ = 50

clip_ = Clip(min_, max_)
im_1_c = clip_.__call__(im_1)

show_image(im_1_c)

In [ ]:
norm_ = Normalize()

im_1_norm = clip_.__call__(im_1)
show_image(im_1_norm)

In [ ]:
stand_ = Standardize()

im_1_std = clip_.__call__(im_1)

show_image(im_1_std)

In [ ]:
flip_ = Flip()

im_1_f = flip_.__call__(im_1)

show_image(im_1_f)

In [ ]:
crop_ = Crop((0.7, 0.7))

im_1_cr = crop_(im_1)

show_image(im_1_cr)

In [ ]:
sharp_ = Sharpen(max_center=25)

im_1_sh = sharp_(im_1)

show_image(im_1_sh)

In [ ]:
max_kernel = (25, 25)
blur_ = GaussianBlur(max_kernel)

im_1_bl = blur_(im_1)

show_image(im_1_bl)

In [ ]:
im_1_br = increase_brightness(im_1, value=50)
show_image(im_1_br)

In [10]:
# ???????????????????
class Contrast(object):
    def __init__(self, range_contrast=(-50, 50)):
        self.range_contrast = range_contrast

    def __call__(self, X):
        contrast = np.random.randint(*self.range_contrast)
        X = X * (contrast / 127 + 1) - contrast
        return X

In [13]:
contr_ = Contrast(range_contrast=(-50, 50))

im_1_contr = contr_(im_1)

show_image(im_1_contr)

In [12]:
uni_noise = UniformNoise(low=-10, high=10)

im_1_uni = uni_noise(im_1)
im_1_uni = im_1_uni.astype(np.uint8)
show_image(im_1_uni)

In [43]:
gaus_noise = GaussianNoise(center=0, std=40)

im_1_gaus = gaus_noise(im_1)
# im_1_uni = im_1_uni.astype(np.uint8)
im_1_gaus = im_1_gaus.astype(np.uint8)
show_image(im_1_gaus)

In [55]:
class LensDistortion(object):
    def __init__(self, d_coef=(0.25, 0.25, 0.1, 0.1, 0.05)):
        self.d_coef = np.array(d_coef)

    def __call__(self, X):

        # get the height and the width of the image
        h, w = X.shape[:2]

        # compute its diagonal
        f = (h ** 2 + w ** 2) ** 0.5

        # set the image projective to carrtesian dimension
        K = np.array([[f, 0, w / 2],
                      [0, f, h / 2],
                      [0, 0,     1]])

        d_coef = self.d_coef * np.random.random(5) # value
        d_coef = d_coef * (2 * (np.random.random(5) < 0.5) - 1) # sign
        # Generate new camera matrix from parameters
        M, _ = cv2.getOptimalNewCameraMatrix(K, d_coef, (w, h), 0)

        # Generate look-up tables for remapping the camera image
        remap = cv2.initUndistortRectifyMap(K, d_coef, None, M, (w, h), 5)

        # Remap the original image to a new image
        X = cv2.remap(X, *remap, cv2.INTER_LINEAR)
        return X

In [52]:
np.random.random(5)

array([0.86533598, 0.80126828, 0.81275173, 0.10935162, 0.66847542])

In [62]:
lens_dist = LensDistortion()
im_1_lens_ = lens_dist(im_1)
show_image(im_1_lens_)

In [87]:
np.random.uniform(-1, 1, 3) * (120, 120, 360)

array([ 69.02197368, -35.13404171, 191.83475049])

In [94]:
class Perspective(object):
    def __init__(self,
                 max_ratio_translation=(0.2, 0.2, 0),
                 max_rotation=(360, 360, 360),
                 max_scale=(0.1, 0.1, 0.2),
                 max_shearing=(15, 15, 5)):

        self.max_ratio_translation = np.array(max_ratio_translation)
        self.max_rotation = np.array(max_rotation)
        self.max_scale = np.array(max_scale)
        self.max_shearing = np.array(max_shearing)

    def __call__(self, X, Y):

        # get the height and the width of the image
        h, w = X.shape[:2]
        max_translation = self.max_ratio_translation * np.array([w, h, 1])
        # get the values on each axis
        t_x, t_y, t_z = np.random.uniform(-1, 1, 3) * max_translation
        r_x, r_y, r_z = np.random.uniform(-1, 1, 3) * self.max_rotation
        sc_x, sc_y, sc_z = np.random.uniform(-1, 1, 3) * self.max_scale + 1
        sh_x, sh_y, sh_z = np.random.uniform(-1, 1, 3) * self.max_shearing

        # convert degree angles to rad
        theta_rx = np.deg2rad(r_x)
        theta_ry = np.deg2rad(r_y)
        theta_rz = np.deg2rad(r_z)
        theta_shx = np.deg2rad(sh_x)
        theta_shy = np.deg2rad(sh_y)
        theta_shz = np.deg2rad(sh_z)


        # compute its diagonal
        diag = (h ** 2 + w ** 2) ** 0.5
        # compute the focal length
        f = diag
        if np.sin(theta_rz) != 0:
            f /= 2 * np.sin(theta_rz)

        # set the image from cartesian to projective dimension
        H_M = np.array([[1, 0, -w / 2],
                        [0, 1, -h / 2],
                        [0, 0,      1],
                        [0, 0,      1]])
        # set the image projective to carrtesian dimension
        Hp_M = np.array([[f, 0, w / 2, 0],
                         [0, f, h / 2, 0],
                         [0, 0,     1, 0]])

        # adjust the translation on z
        t_z = (f - t_z) / sc_z ** 2
        # translation matrix to translate the image
        T_M = np.array([[1, 0, 0, t_x],
                        [0, 1, 0, t_y],
                        [0, 0, 1, t_z],
                        [0, 0, 0,  1]])

        # calculate cos and sin of angles
        sin_rx, cos_rx = np.sin(theta_rx), np.cos(theta_rx)
        sin_ry, cos_ry = np.sin(theta_ry), np.cos(theta_ry)
        sin_rz, cos_rz = np.sin(theta_rz), np.cos(theta_rz)
        # get the rotation matrix on x axis
        R_Mx = np.array([[1,      0,       0, 0],
                         [0, cos_rx, -sin_rx, 0],
                         [0, sin_rx,  cos_rx, 0],
                         [0,      0,       0, 1]])
        # get the rotation matrix on y axis
        R_My = np.array([[cos_ry, 0, -sin_ry, 0],
                         [     0, 1,       0, 0],
                         [sin_ry, 0,  cos_ry, 0],
                         [     0, 0,       0, 1]])
        # get the rotation matrix on z axis
        R_Mz = np.array([[cos_rz, -sin_rz, 0, 0],
                         [sin_rz,  cos_rz, 0, 0],
                         [     0,       0, 1, 0],
                         [     0,       0, 0, 1]])
        # compute the full rotation matrix
        R_M = np.dot(np.dot(R_Mx, R_My), R_Mz)

        # get the scaling matrix
        Sc_M = np.array([[sc_x,     0,    0, 0],
                         [   0,  sc_y,    0, 0],
                         [   0,     0, sc_z, 0],
                         [   0,     0,    0, 1]])

        # get the tan of angles
        tan_shx = np.tan(theta_shx)
        tan_shy = np.tan(theta_shy)
        tan_shz = np.tan(theta_shz)
        # get the shearing matrix on x axis
        Sh_Mx = np.array([[      1, 0, 0, 0],
                          [tan_shy, 1, 0, 0],
                          [tan_shz, 0, 1, 0],
                          [      0, 0, 0, 1]])
        # get the shearing matrix on y axis
        Sh_My = np.array([[1, tan_shx, 0, 0],
                          [0,       1, 0, 0],
                          [0, tan_shz, 1, 0],
                          [0,       0, 0, 1]])
        # get the shearing matrix on z axis
        Sh_Mz = np.array([[1, 0, tan_shx, 0],
                          [0, 1, tan_shy, 0],
                          [0, 0,       1, 0],
                          [0, 0,       0, 1]])
        # compute the full shearing matrix
        Sh_M = np.dot(np.dot(Sh_Mx, Sh_My), Sh_Mz)

        Identity = np.array([[1, 0, 0, 0],
                             [0, 1, 0, 0],
                             [0, 0, 1, 0],
                             [0, 0, 0, 1]])

        # compute the full transform matrix
        M = Identity
#         M = np.dot(Sh_M, M)
        M = np.dot(R_M,  M)
#         M = np.dot(Sc_M, M)
#         M = np.dot(T_M,  M)
#         M = np.dot(Hp_M, np.dot(M, H_M))
        # apply the transformation
        X = cv2.warpPerspective(X, M, (w, h))
        return X

In [98]:
persp = Perspective()
im_1_persp = lens_dist(im_1)
show_image(im_1_persp)

In [108]:
im_1_rot = rotate_func(im_1, angle=-15)
show_image(im_1_rot)

In [114]:
def translate_func(im_1, x_tr=0.1, y_tr=0.1):
    h, w = im_1.shape[:2]
    tx, ty = w*x_tr, h*y_tr
    translation_matrix = np.array([[1, 0, tx],
                                   [0, 1, ty]], dtype=np.float32)
    
    return cv2.warpAffine(im_1, translation_matrix, dsize=(w, h))

In [116]:
im_1_tr = translate_func(im_1, x_tr=0.05, y_tr=-0.05)
show_image(im_1_tr)